<a href="https://colab.research.google.com/github/Jaewon45/ESAA_2022/blob/main/1031_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8. 텍스트분석

## 04. 텍트 분류 실습 - 20 뉴스그룹 분류

- 사이킷런 예제 데이터
- 텍스트 피처 벡터화 
 - 희소 행렬 형태
 - 희소 행렬 분류에 효과적인 알고리즘 : 로지스틱 회귀, SVM, 나이브 베이즈 등

### 텍스트 정규화

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [ ]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [ ]:
import pandas as pd
import numpy as np

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
# 개별 데이터 구성
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 내용을 제외한 다른 정보 제거
- 제목, 소속, 이메일 주소 등 헤더와 푸터 정보들은 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많아 웬만한 ML 알고리즘을 적용해도 높은 예측 성능을 나타냄

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 테스트 데이터에서 CountVectorizer 적용시 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야함
 - 학습시 설정된 피처 개수와 테스트 데이터를 변환한 피처 개수가 같아지도록

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountFectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 테스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 테스트의 CountVectorizer Shape: (11314, 101631)


- 101631개의 피처-단어가 생성됨
- 로지스틱 회귀를 적용해 뉴스그룹 분류 예측

In [ ]:
## 1) Count 기반 예측
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# RogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.607674


In [ ]:
## 2) TF-IDF 기반 예측

from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

# RogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.673659


- TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도 제공
- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법
 1. 최적의 ML 알고리즘 선택
 2. 최상의 피처 전처리 수행

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.692246


In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidt_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}


In [ ]:
# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidt_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701009


### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- Pipeline 클래스를 이용해 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성 한번에 진행
 - 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드 생성 가능
 - 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘 데이터로 입력 가능해 수행 시간 절약

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfid_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
# pipeline의 fit과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701009


- Pipeline 기반에서 하이퍼파라미터 튜닝을 GridSearchCV 방식으로 진행 가능
 - 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터 한번에 최적화 가능
 - 하이퍼파라미터명이 객체 변수명과 결합되어 제공됨 : 개별 객체명과 파라미터/하이퍼파라미터명을 결합해 key값으로 할당

In [ ]:
from sklearn.model_selection import GridSearchCV

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))')

Fitting 3 folds for each of 27 candidates, totalling 81 fits


- SVM, 나이브 베이즈 알고리즘도 희소 행렬 기반의 텍스트 분류에 자주 사용되는 알고리즘